In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-06-20 15:26:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  39.0MB/s    in 3.7s    

2020-06-20 15:26:58 (39.0 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-06-20 15:28:13--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  21.0MB/s    in 0.5s    

2020-06-20 15:28:14 (21.0 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [2]:
import os
import zipfile

In [5]:

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [4]:

train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

train_human_dir = os.path.join('/tmp/horse-or-human/humans')

val_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

val_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [5]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))
print('total validation horse images:', len(os.listdir(val_horse_dir)))
print('total validation human images:', len(os.listdir(val_human_dir)))

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


In [6]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512,activation='relu'),
  tf.keras.layers.Dense(1,activation='sigmoid'),
  
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [13]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dgen = ImageDataGenerator(rescale=1/255)
val_dgen  = ImageDataGenerator(rescale=1/255)

train_generator = train_dgen.flow_from_directory(
    '/tmp/horse-or-human',
    target_size=(300,300),
    batch_size=128,
    class_mode='binary'
)

test_generator = val_dgen.flow_from_directory(
    '/tmp/validation-horse-or-human',
    target_size=(300,300),
    batch_size=32,
    class_mode='binary'
)



Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [14]:
his = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data=test_generator,
    validation_steps=8
)

Epoch 1/15
8/8 [==============================] - 9s 1s/step - loss: 0.7387 - acc: 0.5551 - val_loss: 0.6339 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 6s 809ms/step - loss: 0.6046 - acc: 0.6841 - val_loss: 0.3655 - val_acc: 0.8633
Epoch 3/15
8/8 [==============================] - 7s 870ms/step - loss: 0.2438 - acc: 0.9316 - val_loss: 1.1886 - val_acc: 0.8594
Epoch 4/15
8/8 [==============================] - 6s 778ms/step - loss: 0.4553 - acc: 0.8576 - val_loss: 0.2466 - val_acc: 0.9062
Epoch 5/15
8/8 [==============================] - 6s 708ms/step - loss: 0.2589 - acc: 0.8682 - val_loss: 0.4562 - val_acc: 0.8945
Epoch 6/15
8/8 [==============================] - 7s 874ms/step - loss: 0.1897 - acc: 0.9326 - val_loss: 0.9759 - val_acc: 0.8750
Epoch 7/15
8/8 [==============================] - 6s 777ms/step - loss: 0.1338 - acc: 0.9422 - val_loss: 1.4110 - val_acc: 0.8398
Epoch 8/15
8/8 [==============================] - 6s 798ms/step - loss: 0.1250 - acc: 0.9433 

In [18]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  #images = np.vstack([x])
  classes = model.predict(x)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving Screenshot (8).png to Screenshot (8) (2).png
[0.]
Screenshot (8).png is a horse
